In [2]:
# If not installed, run this cell. It is needed for access to GEE
#!pip install ee
#!pip show ee

In [4]:
# If not installed, run this cell. It helps visualization of data
#!pip install geemap
#!pip show geemap

In [7]:
# Import libraries
import ee, datetime, pandas as pd, geemap

In [6]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-jvg')

In [8]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [6]:
# Access an image asset from my account to test the code
#guate_bdry = ee.FeatureCollection('users/jvg/guatemala_buffer')
#guate_geom = guate_bdry.geometry()
#Map.centerObject(guate_geom)
#Map.addLayer(guate_bdry, {"color": 'red'}, 'Guatemala Boundary')

In [51]:
def determine_period(start_date, end_date, warmup_yrs = 0):
    ini_date = str((int(start_date.split("-")[0]) + int(warmup_yrs))) + "-" + start_date.split("-")[1]
    if (end_date.split("-")[1]) == "12":
        fin_date = str(int(end_date.split("-")[0]) + 1) + "-01"
    else:
        fin_date = str(int(end_date.split("-")[0])) + "-" + str(int(end_date.split("-")[1]) + 1)

    if (datetime.datetime(int(ini_date.split("-")[0]), int(ini_date.split("-")[1]), 1) >= datetime.datetime(int(fin_date.split("-")[0]), int(fin_date.split("-")[1]), 1)):
        if (warmup_yrs != 0):
            print ("Start date is finally set beyond or same as the end date! Please consider the implications of the warm-up years. Remember that end date is exclusive")
        else:
            print ("Start date is finally set beyond or same as the end date! Please review your input dates. Remember that end date is exclusive")
        raise SystemExit()

    print (ini_date, fin_date)
    return ini_date, fin_date

In [52]:
# Initial parameters
warmup_yrs = 0
start_date = '2017-01' # Inclusive
end_date = '2017-12' # Exclusive
ini_date, fin_date = determine_period(start_date, end_date, warmup_yrs)
amazon_basin_id = 6030007000 # This id was identified by using the shapefiles available on the webpage of HydroSHEDS

2017-01 2018-01


In [53]:
# HydroSHEDS v1 will be used. Specifically, its HydroBASINS
hydrobasins = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_3')
zoi = hydrobasins.select("HYBAS_ID").filter(ee.Filter.eq("HYBAS_ID", amazon_basin_id))
visualization = {
  "color": '808080',
  "strokeWidth": 1
}
Map.centerObject(zoi.geometry())
Map.addLayer(zoi, visualization, 'Amazon Basin')

In [54]:
# TerraClimate layers
terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filter(ee.Filter.date(ini_date, fin_date)) # start_date (inclusive), end_date(exclusive)
prec = terraclimate.select('pr')
pet = terraclimate.select('pet') # Scale factor: 0.1
runoff = terraclimate.select('ro')

In [55]:
terraclimate.size().getInfo()

12

In [56]:
# Public link to the asset whc: https://code.earthengine.google.com/?asset=projects/ee-jvg/assets/whc_fc33
whc = ee.Image('projects/ee-jvg/assets/whc_fc33')
whc = whc.clip(zoi)
Map.addLayer(whc, {}, 'Water Holding Capacity')

In [57]:
#############################
# HELPFUL CODES
#############################

# To know how many bands an ImageCollection has
#terraclimate.size().getInfo()

# To get the attrIbutes (columns) as pandas dataframe
#geemap.ee_to_df(hydrobasins)